In [11]:
# Import necessary packages
import os
import glob
import pandas as pd
import numpy as np
from scipy import stats
import scikit_posthocs
import iqplot
import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

# Import data from directory of measurement tables, collected from Fiji

In [14]:
# Define path to directory with measurements
path = os.path.abspath('raw_data_csvs/')
df_summary = pd.DataFrame()
list_summary = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*Summary.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name, then parse experiment details from Image name
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    
#     (df['Date'], df['CellLine'], df['Dose'], df['Time'], df['Treatment'], 
#      df['Channels'], df['Mag'], df['FOV'], df['Measurement']) = zip(*df['Image'].map(lambda x:x.split('_')))

    (df['Date'], df['CellLine'], df['Treatment'], df['Rep'], df['Channels'], 
     df['FOV'], df['Measurement']) = zip(*df['Image'].map(lambda x:x.split('_')))

#     (df['Date'], df['Treatment'], df['FOV'], df['Measurement']) = zip(*df['Image'].map(lambda x:x.split('_')))

    # Drop unnecessary columns for tidiness
    df = df.drop(['Total Area', 'Average Size', '%Area', 'Mean', 'IntDen', 'Image', 'Channels'], axis = 1)
    
    # Compile data
    list_summary.append(df)
df_summary = pd.concat(list_summary, sort=False)


# Preview dataframe to confirm import successful
df_summary.head()

,Slice,Count,Date,CellLine,Treatment,Rep,FOV,Measurement
0,1,19,20200820,U2OS,SMPD3,Rep2,FOV3,PunctaSummary
1,2,19,20200820,U2OS,SMPD3,Rep2,FOV3,PunctaSummary
0,1,5,20200820,U2OS,SMPD3N130A,Rep2,FOV7,PunctaSummary
1,2,8,20200820,U2OS,SMPD3N130A,Rep2,FOV7,PunctaSummary
0,1,18,20200820,U2OS,Dyn1K44A,Rep2,FOV1,PunctaSummary


# Parse dataframe by desired 'cell line' and 'treatment' combinations, then plot results

In [25]:
# Generate and display list of cell lines and treatments present in this dataset
cell_list = df_summary['CellLine'].unique().tolist()
treatment_list = df_summary['Treatment'].unique().tolist()

print('Cells lines: ' + str(cell_list))
print('Treatments: ' + str(treatment_list))

# Save out source data
df_summary = df_summary.sort_values(by=['Treatment'])
df_summary.to_csv('U2OS_Tf633_Puncta_Source_Data.csv')

Cells lines: ['U2OS']
Treatments: ['2aRFP', 'Dyn1K44A', 'SMPD3', 'SMPD3N130A']


In [26]:
# Prepare for parsing data:
# To populate this "comment on/off" code block, copy the results of th cell lists above
cells = [
    'U2OS', 
]
treatments = [
        '2aRFP',
        'Dyn1K44A',
        'SMPD3',
        'SMPD3N130A', 
        ]


from bokeh.palettes import RdBu4

# Copy dataset to not disrupt raw data
df_subset = df_summary

# Pull out only cells and treaments of interest
df_subset = df_subset.loc[df_subset['CellLine'].isin(cells)]
df_subset = df_subset.loc[df_subset['Treatment'].isin(treatments)]
# df_subset = df_subset['Count'].dropna()

# Make ECDF plot using iqplot
data_ecdf = iqplot.ecdf(
    data=df_subset, q='Count', cats='Treatment'
#     ,title=str(cells) + ' cells treated with ' + str(treatments)
    ,style='staircase'
#     ,conf_int=True, n_bs_reps=1000, ptiles=[2.5, 97.5]   # ptiles values equate to 95% CIs
    ,conf_int=True, n_bs_reps=1000, ptiles=[16, 84]   # ptiles values equate to 68% CIs (SEM)
        ,line_kwargs=dict(line_width=3)
    ,show_legend=False

    # Other customization parameters
    ,frame_height = 350, frame_width = 450
    ,order = treatments
    ,palette = ['#1f77b4', '#d62728', '#2ca02c', '#ff7f0e']
#     ,palette = ['#019DE1', '#0070A1', '#00A6ED', '#008BC7']
#     ,palette = ['#E8754F', '#2066AC', '#68ADCC', '#666666']
    ,x_axis_label='Transferrin-633 Puncta Count', y_axis_label='Cumulative Distribution Frequency'
    ,x_range=(0,50)
#     ,y_axis_type='log'
        
)

# Other customization parameters
data_ecdf.axis.axis_label_text_font_size = '20px'
data_ecdf.axis.axis_label_text_font_style = 'normal'
data_ecdf.axis.major_label_text_font_size = '18px'


# data_ecdf.output_backend = "svg"
bokeh.io.show(data_ecdf)

In [41]:
# Prepare for parsing data:
# To populate this "comment on/off" code block, copy the results of th cell lists above
cells = [
    'U2OS', 
]
treatments = [
        '2aRFP',
        'Dyn1K44A',
        'SMPD3',
        'SMPD3N130A', 
        ]


from bokeh.palettes import RdBu4

# Copy dataset to not disrupt raw data
df_subset = df_summary

# Pull out only cells and treaments of interest
df_subset = df_subset.loc[df_subset['CellLine'].isin(cells)]
df_subset = df_subset.loc[df_subset['Treatment'].isin(treatments)]
# df_subset = df_subset['Count'].dropna()

# Make ECDF plot using iqplot
data_stripbox = iqplot.stripbox(
    data=df_subset, q='Count', cats='Treatment', q_axis='y'
    
    # Other customization parameters
    ,frame_height = 300, frame_width = 300
    ,order = treatments
    ,palette = ['#1f77b4', '#d62728', '#2ca02c', '#ff7f0e']
    ,jitter=True
    ,x_axis_label='Treatment', y_axis_label='Transferrin-633 Puncta'
    ,marker_kwargs=dict(size=5, alpha=0.7)
    ,top_level='box'
    ,box_kwargs=dict(line_width=2, line_color='black')
    ,whisker_kwargs=dict(line_width=2, line_color='black')
    ,median_kwargs=dict(line_width=2, line_color='black')
)

# Other customization parameters
data_stripbox.title.text_font_size = '18px'
data_stripbox.legend.title='Treatment'
# data_ecdf.legend.title_text_font_style = 'bold'
data_stripbox.legend.title_text_font_size = '16px'
data_stripbox.legend.label_text_font_size = '14px'

data_stripbox.axis.axis_label_text_font_size = '16px'
data_stripbox.axis.axis_label_text_font_style = 'bold'
data_stripbox.axis.major_label_text_font_size = '14px'

# data_stripbox.output_backend = "svg"
bokeh.io.show(data_stripbox)

# Kruskal-Wallis Test with Dunn's Multiple Comparisons Correction
Useful for comparing multiple datasets

In [ ]:
# Reminder of treatments to compare
treatment_list

['2aRFP', 'Dyn1K44A', 'SMPD3', 'SMPD3N130A']

In [ ]:
### Kruskal-Wallis test
# Define samples to compare
sample1 = '2aRFP'
sample2 = 'Dyn1K44A'
sample3 = 'SMPD3'
sample4 = 'SMPD3N130A'


# Run Kruskal-Wallis test
kw_result = stats.kruskal(
                df_subset.loc[df_subset['Treatment']==sample1]['Count']
               ,df_subset.loc[df_subset['Treatment']==sample2]['Count']
               ,df_subset.loc[df_subset['Treatment']==sample3]['Count']
               ,df_subset.loc[df_subset['Treatment']==sample4]['Count']
#                ,df_subset.loc[df_subset['Treatment']==sample5]['Count']
#                ,df_subset.loc[df_subset['Treatment']==sample6]['Count']
    )

# Dunn's Posthoc for Multiple Comparisons
mult_compar = scikit_posthocs.posthoc_dunn(df_subset, val_col='Count', group_col='Treatment'
                                           , sort=False, p_adjust='bonferroni').round(6)

# Display test results
print('Kruskal-Wallis test results: \n\t\t\t\t statistic=' + str(kw_result[0]) + 
    '\n\t\t\t\t p-value=' + str(kw_result[1]))
print("\nDunn's posthoc multiple comparison result: \n" + str(mult_compar) +'\n')
# mult_compar.to_csv("Results of Dunn's Posthoc.csv")

# Get number of cells within this test
for treatment in df_subset['Treatment'].unique().tolist():
    temp_df = df_subset.loc[df_subset['Treatment'] == treatment]
    print('n = ' + str(len(temp_df)) + ' cells in the ' + str(treatment) + ' dataset.')

Kruskal-Wallis test results: 
				 statistic=42.81823141572586
				 p-value=2.689693437699311e-09

Dunn's posthoc multiple comparison result: 
               2aRFP  Dyn1K44A     SMPD3  SMPD3N130A
2aRFP       1.000000  0.000048  0.047485    1.000000
Dyn1K44A    0.000048  1.000000  0.000000    0.000700
SMPD3       0.047485  0.000000  1.000000    0.034371
SMPD3N130A  1.000000  0.000700  0.034371    1.000000

n = 50 cells in the 2aRFP dataset.
n = 35 cells in the Dyn1K44A dataset.
n = 29 cells in the SMPD3 dataset.
n = 36 cells in the SMPD3N130A dataset.
